In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Load data from Excel file
#PT_BR = Carrega a base de volumes a serem optimizados
#ENG = Load volume flow that should be optimized
arquivo_excel = 'optimizacao.xlsx'
aba_volumes = 'dados'
planilha1 = pd.read_excel(arquivo_excel, sheet_name=aba_volumes)



#PT_BR = Carrega a relação de centros de distribuição e a relação de clientes \ pontos de entrega
#ENG = Load DC IDs and Costumer \ Delivery Point IDs
id_cds = planilha1['CDs'].unique()
id_clientes = planilha1['Clientes'].unique()

#PT_BR = Carrega as distancias entre os centros de distribuição e a fabrica. Todos para Todos
#ENG = Load distances between DCs and factories. All to All.

dist_dc_data = pd.read_excel(arquivo_excel, sheet_name='dist_fab_CDS')
dados_dc_dist = dist_dc_data['Distancia'].to_numpy()

#PT_BR= Define o tamanho da modelagem
#ENG= Define modeling size
num_cds = len(id_cds)
num_clientes = len(id_clientes)


#PT_BR=função objetivo
#ENG=Objective Function
funcao_c = np.zeros(num_cds * num_clientes)
for i in range(num_clientes):
    for j in range(num_cds):
        dist_fabrica_CD = dados_dc_dist[j]
        dist_CD_cliente=planilha1.loc[(planilha1['Clientes'] == id_clientes[i]) & (planilha1['CDs'] == id_cds[j]), 'distancia'].values[0]
     
        c[i * num_cds + j] = dist_CD_cliente+dist_fabrica_CD
#PT_BR= Define o volume máximo de interações a serem testadas
#ENG=Define the maximum of scenarios
funcao_a = np.zeros((num_clientes, num_cds * num_clientes))
#PT_BR=Define o numero de resultados distintos (Total de clientes)
#ENG=Define the number of results (Costumers)
funcao_b = np.ones(num_clientes)

for i in range(num_clientes):
    for j in range(num_cds):
        funcao_a[i, i * num_cds + j] = 1
#PT_BR=Limites que considera como binário,ou seja, somente fluxos com dados
#ENG=Bound that define all flows with data.
bounds_malha = [(0, 1)] * (num_cds * num_clientes)

#PT_BR=Executa a optimização.
#ENG=Solve the problemn
#Executar modelagem
print("Executando modelagem")
arquivo_resultado='resultado_malha.xlsx'
with pd.ExcelWriter(arquivo_resultado,engine='openpyxl') as writer:
    resultado = linprog(funcao_c, A_ub=funcao_a, b_ub=funcao_b, bounds=bounds_malha)

    # Abertura dos resultados
    #PT_BR=Abertura do resultado
    #ENG=Results by Costumer/DC
    viagens_por_cd = resultado.x[:num_cds * num_clientes]
    viagens_por_cliente = np.reshape(viagens_por_cd, (num_clientes, num_cds))

    melhores_cds = [id_cds[np.argmax(viagens_por_cliente[i])] for i in range(num_clientes)]
    resultados_df=pd.DataFrame({'Cliente_ID':id_clientes,'Melhor CD':melhores_cds})

    #PT_BR=Exporta a relação de clientes e os melhores CDS considerando o fluxo total (fabrica->CD->Cliente)
    resultados_df.to_excel(writer,sheet_name="malha_optimizada",index=False)
    writer.save()
    print('Modelagem completa com sucesso')
    
